In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
import seaborn as sn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()

from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as sm2
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path = '/Volumes/Cisco/Fall2021/quantum-revision/Analysis/RQ3/regression-cleaned2/'

In [3]:
file = 'debt_files_details_tags.xlsx'
xl1 = pd.ExcelFile(path + file)
df_data = xl1.parse('debt_files_details_tags')

In [4]:
df_data.head()

,Category,project,snapshot_id,FileId,total_debt,total_effort,bug_inducing,bug_fixing,inducing_loc,fixing_loc,...,error_handling,lock_in,accessibility,brain_overload,convention,unpredictable,design,api_design,redundant,confusing
0,Cryptography,theQRL/QRL,13,2,309,2237,213,101,20153,2018,...,32,0,0,34,33,0,17,0,14,0
1,Cryptography,theQRL/QRL,13,3,299,1620,42,12,4884,315,...,3,0,0,9,122,0,0,0,31,9
2,Cryptography,theQRL/QRL,13,4,1029,3229,267,102,23596,3841,...,39,0,0,52,507,0,34,0,125,1
3,Cryptography,theQRL/QRL,13,5,61,267,30,7,1323,50,...,7,0,0,1,25,0,3,0,10,0
4,Cryptography,theQRL/QRL,13,6149,1,5,3,1,68,31,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset

In [6]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91700 entries, 0 to 91699
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Category         91700 non-null  object 
 1   project          91700 non-null  object 
 2   snapshot_id      91700 non-null  int64  
 3   FileId           91700 non-null  int64  
 4   total_debt       91700 non-null  int64  
 5   total_effort     91700 non-null  int64  
 6   bug_inducing     91700 non-null  int64  
 7   bug_fixing       91700 non-null  int64  
 8   inducing_loc     91700 non-null  int64  
 9   fixing_loc       91700 non-null  int64  
 10  total_loc        91700 non-null  int64  
 11  sonar_loc        91700 non-null  float64
 12  is_inducing      91700 non-null  int64  
 13  is_debt          91700 non-null  int64  
 14  ERROR            91700 non-null  int64  
 15  CODE_SMELL       91700 non-null  int64  
 16  is_erroneous     91700 non-null  int64  
 17  is_smelly   

In [19]:
df_data2 = df_data.drop(columns=['Category', 'project','FileId','snapshot_id','sonar_loc', 'inducing_loc',
                                 'bug_fixing','is_inducing','fixing_loc'])
bug_inducing = df_data.bug_inducing.values
snapshot_id = df_data.snapshot_id.values
bug_fixing = df_data.bug_fixing.values
sonar_loc = df_data.sonar_loc.values
total_loc = df_data.total_loc.values
is_inducing = df_data.is_inducing.values
fixing_loc = df_data.fixing_loc.values

In [20]:
empty_columns = []
for column in df_data2.columns:
    if np.sum(df_data[column].values.tolist()) == 0:
        empty_columns.append(column)
df_data2 = df_data2.drop(columns=empty_columns)
print('droped columns: ',empty_columns)

droped columns:  ['wcag2_a', 'based_on_misra', 'finding', 'psr2', 'duplicate', 'lock_in', 'unpredictable']


In [21]:
df_data2.head()

,total_debt,total_effort,bug_inducing,total_loc,is_debt,ERROR,CODE_SMELL,is_erroneous,is_smelly,MAJOR,...,suspicious,obsolete,error_handling,accessibility,brain_overload,convention,design,api_design,redundant,confusing
0,309,2237,213,62084,1,10,299,1,1,3,...,11,0,32,0,34,33,17,0,14,0
1,299,1620,42,13526,1,11,288,1,1,1,...,6,0,3,0,9,122,0,0,31,9
2,1029,3229,267,75417,1,8,1021,1,1,80,...,49,0,39,0,52,507,34,0,125,1
3,61,267,30,1855,1,2,59,1,1,3,...,3,0,7,0,1,25,3,0,10,0
4,1,5,3,130,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_data_1 = df_data2.copy()

In [33]:
def get_features(df_, removed_features):
    index = 0
    cloc = df_.total_loc.values.tolist()
    for col in df_.columns:
        if not 'is_debt' in col and not 'is_erroneous' in col and not 'is_smelly' in col:
            #if not 'is_bug' in col:
            kloc = cloc[index]
            if kloc == 0:
                kloc = 1
            df_[col] = df_[col]/ kloc
        index += 1
    #ERROR+CODE_SMELL+MAJOR+CRITICAL+MINOR+BLOCKER
    feature_columns = ['user_experience','bad_practice','suspicious','design','redundant', 'python3', 'obsolete', 'clumsy', 'accessibility', 'pitfall', 'unused', 'brain_overload', 'error_handling', 'cwe', 'convention','confusing']
    feature_str = ''
    for col_name in feature_columns:
        if not col_name in removed_features:
            if col_name != 'confusing':
                feature_str += col_name+'+'
            else:
                feature_str += col_name
    if feature_str.endswith('+'):
        feature_str = feature_str[0:len(feature_str)-1]
    #y2, X2 = dmatrices('bug_inducing ~ is_erroneous+is_smelly+ERROR+CODE_SMELL+user_experience+bad_practice+suspicious+design+redundant+python3+obsolete+clumsy+accessibility+pitfall+unused+brain_overload+error_handling+cwe+convention+confusing', df_, return_type='dataframe')
    y2, X2 = dmatrices('bug_inducing ~ is_erroneous+is_smelly+'+feature_str, df_, return_type='dataframe')
    
    #y2, X2 = dmatrices('bug_inducing ~ is_smelly+is_bug+accessibility+brain_overload+unused+confusing+cwe+suspicious+redundant+error_handling+design+python3+bad_practice+obsolete+clumsy+user_experience', df_, return_type='dataframe')
    vif2 = pd.DataFrame()
    vif2["VIF"] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
    vif2["features"] = X2.columns
    
    features = ''
    columns = []
    for i in range(1,len(vif2.VIF.values)):
        if vif2.VIF.values[i] <= 2.5:
            if i < len(vif2.VIF.values)-1:
                features = features+vif2.features.values[i]+'+'
                columns.append(vif2.features.values[i])
            else:
                features = features+vif2.features.values[i]
                columns.append(vif2.features.values[i])
    if features[-1:] == "+":
        features = features[:-1]
    return features, columns, df_ 

In [34]:
def ols_regr(df, removed_columns):
    features, columns, df_ = get_features(df, removed_columns)
    #result = sm2.ols(formula="bug_inducing ~ Deffect+Smells+Defect_effort+Critical+Major+Minor+Blocker+Critical_effort+accessibility+brain_overload+unused+confusing+convention+cwe+suspicious+redundant+error_handling+design+api_design+python3+pitfall+bad_practice+obsolete+clumsy+user_experience", data=df).fit()
    #result = sm2.ols(formula="bug_inducing ~ accessibility+brain_overload+unused+confusing+convention+cwe+suspicious+redundant+error_handling+design+python3+pitfall+bad_practice+obsolete+clumsy+user_experience", data=df).fit()
    #print(df_.head())
    #print(df_.head())
    result = sm2.ols(formula="bug_inducing ~ {}".format(features), data=df_).fit()
    
    y = df.bug_inducing.values
    X = df[columns].to_numpy()
    
    regressor = LinearRegression()
    regressor.fit(X, y)
    corr = regressor.coef_
    print("Linear Regression coef")
    for i in range(len(columns)):
        print("{}: {}".format(columns[i], corr[i]))
    return result

In [35]:
print(ols_regr(df_data_1, empty_columns).summary())

Linear Regression coef
is_erroneous: 5.080823195282624e-08
is_smelly: 5.157849407021186e-08
user_experience: -1.55531446383058e-09
bad_practice: -3.5921303887679863e-10
suspicious: -4.330848098882132e-09
design: 6.474315759391028e-13
redundant: 6.2898224539583275e-06
python3: -6.928248058178017e-16
obsolete: -1.3388951710170458e-08
clumsy: 3.4615413755564804e-14
accessibility: -3.4133521352937697e-07
pitfall: 4.323555540383979e-09
unused: -2.0048671827465082e-14
brain_overload: 2.5842789684567246e-07
error_handling: 3.74410885698985e-13
cwe: -3.222468806064405e-14
convention: 1.4382741914947145e-07
confusing: -0.0010851152779441176
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     401.0
Date:                Sat, 04 Dec 2021   Prob

In [38]:
def df_sheet(sheetName):
    df_temp = xl1.parse(sheetName)
    df_temp2 = df_temp.drop(columns=['Category', 'project','FileId','snapshot_id','sonar_loc', 'inducing_loc',
                                 'bug_fixing','is_inducing','fixing_loc'])
    
    temp_empty_columns = []
    for column in df_temp2.columns:
        if np.sum(df_temp2[column].values.tolist()) == 0:
            temp_empty_columns.append(column)
    df_temp2 = df_temp2.drop(columns=temp_empty_columns)
    #print('droped columns: ',temp_empty_columns)
    df_temp_1 = df_temp2.copy()
    #print the results
    print('********************************************')
    print('Category: {}'.format(sheetName))
    print('********************************************')
    print(ols_regr(df_temp_1, temp_empty_columns).summary())

In [39]:
df_sheet('Full-stack Library')

********************************************
Category: Full-stack Library
********************************************
Linear Regression coef
is_erroneous: 0.20993642170534096
is_smelly: 0.11190694115317362
user_experience: -0.5496690961765918
suspicious: 0.3625792107920223
design: 0.1032574779352646
redundant: 1.8588654170903625
python3: -6.3551162022049015
clumsy: 0.03861046650530976
accessibility: -1.1090895980255255
unused: -0.11579231505701584
brain_overload: 0.7769958956544768
error_handling: 0.006933354281555619
cwe: -0.30906254747069806
convention: 0.07769559453668101
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     151.2
Date:                Sat, 04 Dec 2021   Prob (F-statistic):               0.00
Time:                 

In [40]:
df_sheet('ToolKit')

********************************************
Category: ToolKit
********************************************
Linear Regression coef
is_erroneous: 0.001247862116480198
is_smelly: 0.0012691621488184397
user_experience: -0.05920244765990735
suspicious: -0.002325149978717523
design: 0.018605296296586273
redundant: 0.03840975111112919
python3: -9.090427705828825e-05
obsolete: -0.09895561046376684
clumsy: 0.16683795719013367
accessibility: -0.025556781401918544
unused: 0.0017090268146356914
error_handling: -0.000858101545149401
cwe: 0.005196826553158158
convention: 0.008537301397892239
confusing: 0.02679171818699429
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     158.1
Date:                Sat, 04 Dec 2021   Prob (F-statistic):        

In [41]:
df_sheet('Experimentation')

********************************************
Category: Experimentation
********************************************
Linear Regression coef
is_erroneous: 0.004113835718578806
is_smelly: 0.002482434901526565
suspicious: 0.0023285013059825546
design: 0.002334886650844694
redundant: -0.036637315123167584
python3: -0.0012813088950639076
clumsy: 0.044181346075550756
unused: 0.016623534861600835
brain_overload: 0.08137544216133409
error_handling: 0.0049836730784338324
cwe: 0.023255671059222084
convention: -1.960302817667972e-05
confusing: 0.02828023883517847
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     110.4
Date:                Sat, 04 Dec 2021   Prob (F-statistic):          6.66e-249
Time:                        19:23:05   Log-Lik

In [42]:
df_sheet('Simulator')

********************************************
Category: Simulator
********************************************
Linear Regression coef
is_erroneous: -0.00860018778692271
is_smelly: 0.09374137560423834
user_experience: 1.1301271991806727
suspicious: 0.17315776369565267
design: 0.09754978885908325
redundant: 0.7558841035443569
python3: -0.033665746883161804
clumsy: 6.5467616494713
accessibility: -0.0005248730435877674
pitfall: 0.19299690203397
unused: 0.04962191511721112
error_handling: 0.03857694784255694
cwe: 4.224321353939941
convention: 0.047520493054745705
confusing: 0.4397720924887552
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     150.4
Date:                Sat, 04 Dec 2021   Prob (F-statistic):               0.00
Time:      

In [43]:
df_sheet('Cryptography')

********************************************
Category: Cryptography
********************************************
Linear Regression coef
is_erroneous: 0.00031650393607363705
is_smelly: 0.00017358155607335638
suspicious: 5.2443735480968025e-05
design: 1.0109429902794523e-05
clumsy: 0.0003926986727867558
pitfall: -0.004562353235040191
error_handling: 0.0005284852764710637
cwe: 0.00010347656484517473
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.555
Model:                            OLS   Adj. R-squared:                  0.554
Method:                 Least Squares   F-statistic:                     532.3
Date:                Sat, 04 Dec 2021   Prob (F-statistic):               0.00
Time:                        19:23:57   Log-Likelihood:                 26871.
No. Observations:                3422   AIC:                        -5.372e+04
Df Residuals:                    3413   BIC:   

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [44]:
df_sheet('Compiler')

********************************************
Category: Compiler
********************************************
Linear Regression coef
is_erroneous: 0.02347872881882387
is_smelly: 0.041046130905080094
redundant: 2.5010257229459403
pitfall: -1.3843858131392837
error_handling: 0.035469747509176586
convention: 0.07071880947412648
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     47.59
Date:                Sat, 04 Dec 2021   Prob (F-statistic):           7.04e-56
Time:                        19:24:18   Log-Likelihood:                 6213.0
No. Observations:                2856   AIC:                        -1.241e+04
Df Residuals:                    2849   BIC:                        -1.237e+04
Df Model:                           6     

In [45]:
df_sheet('Quantum Algorithsm')

********************************************
Category: Quantum Algorithsm
********************************************
Linear Regression coef
is_erroneous: 0.007600437685373435
is_smelly: 0.025472662745395953
user_experience: -1.2177361586582405
suspicious: -0.04747556678619548
design: 0.0663612625816888
redundant: 0.37114579812805143
python3: -0.015543936751448006
clumsy: 0.002287723098627248
unused: -0.09951552461379419
brain_overload: 0.17728218167251414
error_handling: 0.08646966383330525
cwe: -0.001271442041016358
convention: 0.020766351831934194
confusing: 0.20314832991968654
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.198
Method:                 Least Squares   F-statistic:                     87.41
Date:                Sat, 04 Dec 2021   Prob (F-statistic):          1.35e-224
Time:           

In [46]:
df_sheet('Quantum-Chemistry')

********************************************
Category: Quantum-Chemistry
********************************************
Linear Regression coef
is_erroneous: 0.14545729440660202
is_smelly: 0.06322830069500579
user_experience: -0.23075684394813847
suspicious: -0.5364237902327658
design: 0.0006925587648102319
redundant: 2.1132222245440437
python3: -0.028719637527122824
clumsy: 0.5092404449833718
accessibility: -0.004345169093351305
unused: 0.24596136647144395
error_handling: 0.02442024079398409
cwe: -0.029779002445011783
convention: -0.020126116299130904
confusing: 1.1108581437423142
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     71.97
Date:                Sat, 04 Dec 2021   Prob (F-statistic):          5.73e-204
Time:              

In [47]:
df_sheet('Assembly')

********************************************
Category: Assembly
********************************************
Linear Regression coef
is_erroneous: 0.008464146000734444
is_smelly: 0.007215723621031147
user_experience: -0.06363605303203135
suspicious: -0.0019546904739997863
design: 0.0006309415441273975
redundant: -0.0005633308374518812
python3: 0.3527244748216156
clumsy: -0.172196311626182
accessibility: -0.00013149591857733445
unused: 0.00867855197778073
brain_overload: 0.1365438559931432
error_handling: 0.020419586094053015
confusing: -0.013396618198767453
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     23.94
Date:                Sat, 04 Dec 2021   Prob (F-statistic):           3.99e-45
Time:                        19:26:06   Lo

In [48]:
df_sheet('Quantum Annealing')

********************************************
Category: Quantum Annealing
********************************************
Linear Regression coef
is_erroneous: 0.0138458819073786
is_smelly: 0.008430992513422131
suspicious: 0.33757623924764335
design: -0.02670613720517647
redundant: 0.02829677375507509
python3: -0.01221004067868105
clumsy: 0.3138331886924906
unused: 0.012582654724341658
brain_overload: 0.22495352905836838
error_handling: -0.23338612649948753
cwe: -0.16883811016301514
convention: -0.036704810180911
confusing: 0.24839669597088518
                            OLS Regression Results                            
Dep. Variable:           bug_inducing   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     16.21
Date:                Sat, 04 Dec 2021   Prob (F-statistic):           8.13e-34
Time:                        19:26:27   Log-Likelihood:     